# Mundo 14 - Animações com Matplotlib

O matplotlib permite que você crie vídeos a partir dos gráficos.
Um gráfico nada mais é do que uma "foto". Caso você plote um gráfico de linha, por exemplo, a cada ponto, gerando essa "foto", você terá um vídeo no final.

O submódulo que iremos utilizar é o *FuncAnimation*

In [ ]:
%matplotlib notebook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib.ticker as mtick
import numpy as np
from datetime import date
import matplotlib.dates as mdate
import yfinance as yf   
from datetime import datetime
import os
import mplcyberpunk

In [ ]:
params = {"ytick.color" : "k",
          "xtick.color" : "k",
          "axes.labelcolor" : "k",
          "axes.edgecolor" : "k"}
plt.rcParams.update(params)

In [ ]:
cotacoes = yf.download(["^BVSP"], "2001-12-28", "2023-01-01")['Adj Close']

In [ ]:
fig, ax = plt.subplots()

ax.plot(cotacoes)

In [ ]:
cotacoes_ani = cotacoes.resample("M").last()


fig, ax = plt.subplots()
ax.set_xlim(date(2001, 1, 1), date(2023, 12, 31))
ax.set_ylim(6000, 130000)


def animate(i):
    
    data = cotacoes_ani.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "b")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(cotacoes_ani)), interval=50, repeat = False)

plt.show()


In [ ]:
cotacoes_ani.iloc[:int(2+1)]

In [ ]:
dados_iphone = pd.read_csv("dados_iphone.csv", index_col="lancamento")

serie_preco_iphone = dados_iphone['quantidade_salario_minimo']

serie_preco_iphone = serie_preco_iphone.str.replace(",", ".")

serie_preco_iphone = serie_preco_iphone.astype(float)

serie_preco_iphone.index = pd.to_datetime(serie_preco_iphone.index)

serie_preco_iphone

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(date(2008, 1, 1), date(2022, 12, 31))
ax.set_ylim(3, 16)


def animate(i):
    
    data = serie_preco_iphone.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "b")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(serie_preco_iphone)), interval=150, repeat = False)


plt.show()

In [ ]:
#isso aqui aumenta o numero de frames. Basta passa o eixo x, y e o numero de steps que você deseja a mais. 
#Por exemplo, num dataset de 10 itens, caso numsteps = 10, o dataset final terá 100 itens

def augment(xold,yold,numsteps):
    xnew = []
    ynew = []
    for i in range(len(xold)-1):
        difX = xold[i+1]-xold[i]
        stepsX = difX/numsteps
        difY = yold[i+1]-yold[i]
        stepsY = difY/numsteps
        for s in range(numsteps):
            xnew = np.append(xnew,xold[i]+s*stepsX)
            ynew = np.append(ynew,yold[i]+s*stepsY)
    return xnew,ynew

In [ ]:
#essa função suaviza o gráfico, deixando as transições mais arredondadas. 

def smoothListGaussian(listin, degree=5):  
    window=degree*2-1  
    weight=np.array([1.0]*window)  
    weightGauss=[] 
    for i in range(window):  
        i=i-degree+1  
        frac=i/float(window)  
        gauss=1/(np.exp((4*(frac))**2))  
        weightGauss.append(gauss)
    weight=np.array(weightGauss)*weight  
    smoothed=[0.0]*(len(listin)-window)  
    for i in range(len(smoothed)):        
        smoothed[i]=sum(np.array(listin[i:i+window])*weight)/sum(weight)  
    return smoothed

In [ ]:
novos_dadosx_iphone, novos_dadosy_iphone = augment(serie_preco_iphone.index, 
                                                   serie_preco_iphone, numsteps=20)

novos_dadosy_iphone

In [ ]:
dadosy_suavizado_iphone = smoothListGaussian(novos_dadosy_iphone)

dadosy_suavizado_iphone

In [ ]:
print(len(dadosy_suavizado_iphone))
print(len(novos_dadosy_iphone))
print(len(novos_dadosx_iphone))

In [ ]:
novo_indice = pd.date_range(start=serie_preco_iphone.index[0], 
                            end=serie_preco_iphone.index[-1], periods=len(dadosy_suavizado_iphone))

serie_preco_iphone = pd.Series(data=dadosy_suavizado_iphone, index=novo_indice)

serie_preco_iphone

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(date(2008, 1, 1), date(2022, 12, 31))
ax.set_ylim(3, 16)
ax.set_title("Preço do Iphone x Salário Mínimo")


def animate(i):
    
    data = serie_preco_iphone.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "b")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(serie_preco_iphone)), interval=20, repeat = False)


plt.show()

In [ ]:
inflacao_americana = pd.read_csv("CPIAUCSL.csv", index_col="DATE")

taxa_de_juros_americana = pd.read_csv("DFF.csv", index_col="DATE")

taxa_de_juros_americana = taxa_de_juros_americana.squeeze()

taxa_de_juros_americana = taxa_de_juros_americana.str.replace(",", ".")

taxa_de_juros_americana = taxa_de_juros_americana.astype(float)

inflacao_americana = inflacao_americana.squeeze()

inflacao_americana = inflacao_americana.str.replace(",", ".")

inflacao_americana = inflacao_americana.astype(float)

inflacao_americana.index = pd.to_datetime(inflacao_americana.index)

taxa_de_juros_americana.index = pd.to_datetime(taxa_de_juros_americana.index)

inflacao_americana = inflacao_americana.resample("1Q").last()
taxa_de_juros_americana = taxa_de_juros_americana.resample("1Q").last()

inflacao_americana = inflacao_americana[inflacao_americana.index > "1960-12-31"]
taxa_de_juros_americana = taxa_de_juros_americana[taxa_de_juros_americana.index > "1960-12-31"]

var_inflacao_americana = (inflacao_americana.pct_change(periods=4)) * 100
var_inflacao_americana = var_inflacao_americana.dropna()

juro_real = taxa_de_juros_americana - var_inflacao_americana
juro_real = juro_real.dropna()
juro_real = juro_real.astype(float)

juro_real

In [ ]:
fig, ax = plt.subplots()
ax.set_xlim(date(1962, 1, 1), date(2022, 12, 31))
ax.set_ylim(-10, 20)


def animate(i):
    
    data = juro_real.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "b")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(juro_real)), interval=50, repeat = False)


plt.show()

In [ ]:
novo_indice = pd.date_range(start=juro_real.index[0], end=juro_real.index[-1], periods=len(smoothListGaussian(juro_real)))
juro_real = pd.Series(data=smoothListGaussian(juro_real), index=novo_indice)

juro_real

In [ ]:
!pip install ffmpeg

https://www.gyan.dev/ffmpeg/builds/
<br>

https://ffmpeg.org/download.html#build-windows

In [ ]:
plt.style.use("cyberpunk")

fig, ax = plt.subplots()
ax.set_xlim(date(1962, 1, 1), date(2022, 12, 31))
ax.set_ylim(-10, 20)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals = 0))
ax.set_title("Taxa de juros real EUA")

def animate(i):
    
    data = juro_real.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "#00FFFF")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(juro_real)), interval=50, repeat = False)

plt.rcParams['animation.ffmpeg_path'] ='C:\\Users\\lsiqu\\Downloads\\ffmpeg\\bin\\ffmpeg.exe'
writervideo = animation.FFMpegWriter(fps=60) 
ani.save("juro_real_americano.mp4", writer=writervideo)

plt.show()

# Exercícios

* Exercício 108: Crie uma animação do desempenho do preço do dólar desde 2008.
* Exercício 109: Crie uma animação com a correlação móvel do ibovespa x WEGE3 e VALE3 entre 2010 e 2014.

Use as funções de escala e normalização caso ache necessário.
Não esqueça de dar resample nos dados. Dado diário geralmente é ruim pra animação.

In [ ]:
#gabarito 108

dolar = yf.download(["USDBRL=X"], "2007-12-28", "2023-01-01")['Adj Close']

dolar = dolar.resample("M").last()

plt.style.use("cyberpunk")

fig, ax = plt.subplots()
ax.set_xlim(date(2008, 1, 1), date(2022, 12, 31))
ax.set_ylim(0.5, 6)
ax.yaxis.set_major_formatter('R${x:1.0f}')
ax.set_title("Dólar x Real")

def animate(i):
    
    data = dolar.iloc[:int(i+1)] #select data range
    ax.plot(data.index, data.values, color = "#00FFFF")
    
    
ani = FuncAnimation(fig, animate, frames = range(0, len(dolar)), interval=50, repeat = False)

plt.show()

In [ ]:
acoes = ["WEGE3.SA", "VALE3.SA", "^BVSP"]

cotacoes = yf.download(acoes, "2009-01-01", "2015-01-01")['Adj Close']

In [ ]:
#gabarito 109

acoes = ["WEGE3.SA", "VALE3.SA", "^BVSP"]

retornos = cotacoes.pct_change().dropna()

acoes.remove("^BVSP")

janela_cor_weg = retornos['WEGE3.SA'].rolling(252).corr(retornos["^BVSP"]).dropna()
janela_cor_vale = retornos['VALE3.SA'].rolling(252).corr(retornos["^BVSP"]).dropna()

janela_cor_weg = janela_cor_weg.resample("M").mean()
janela_cor_vale = janela_cor_vale.resample("M").mean()

fig, ax = plt.subplots()
ax.set_xlim(date(2010, 1, 1), date(2015, 1, 1))
ax.set_ylim(0, 1)
ax.set_title("Janela de correlação 12M x Ibovespa", fontweight ="bold")


def animate(i):
    
    data = janela_cor_weg.iloc[:int(i+1)] 
    weg = ax.plot(data.index, data.values, color = "#00FFFF", label = "WEGE3")
    data = janela_cor_vale.iloc[:int(i+1)] 
    vale = ax.plot(data.index, data.values, color = "fuchsia", label = "VALE3")
    
    if i == 0:
        ax.legend(["WEGE3", "VALE3"])


ani = FuncAnimation(fig, animate, frames = range(0, len(dolar)), interval=30, repeat = False)

plt.show()




# Mundo 15

Gráfico especiais - Mapa de calor Seaborn